#### Пайплайн для классификации новых текстов (модели из тетрадок 2,4,5)

In [1]:
import re
import pymystem3
import numpy as np
import pandas as pd

In [2]:
from utils import *

In [3]:
from catboost import CatBoostClassifier

In [4]:
def clean(s):
    s = re.sub('\n', ' ', s)
    s = re.sub('<\?xml.+\?>', ' ', s)
    s = re.sub('<\/?[A-Za-z]*>', ' ', s)
    s = re.sub('[0-9]+(\.|,)?[0-9]*', ' num ', s)
    s = re.sub('([a-zа-яёй])([A-ZА-ЯЁЙ])', '\g<1> \g<2>', s)
    s = re.sub('[^A-Za-zА-ЯЁЙа-яёй0-9\s\-]', ' ', s)
    return re.sub('\s{2,}', ' ', s).strip().lower()

def tokenize(s):
    return re.findall('[.,;:!?()"]|\w+-\w+|\w+\'\w+|\w+|-|\'', s)

def lemmatize(words, stem):
    return [stem.lemmatize(w)[0].strip() for w in words]

def remove_stopwords(words, stopwords):
    return [w for w in words if w not in set(stopwords)]

def preprocess(text, stem, stopwords):
    words = lemmatize(tokenize(clean(text)), stem)
    return ' '.join(remove_stopwords(words, stopwords))

* *ничем не отличаются от функций из 1ой тетрадки*
* *не стал раскидывать код по модулям (комментарии к коду в тетрадках)*

In [5]:
def predict_proba(title, text, model, stem, stopwords):
    
    text = preprocess(text, stem, stopwords)
    title = preprocess(title, stem, stopwords)
    
    df = pd.DataFrame({
        'title': [title], 'text': [text], 
        'title_len': [len(title)], 'text_len': [len(text)] 
    })
    
    return np.round(model.predict_proba(df), 3)

* *функция получения распределения вероятностей для одного примера (модели из тетрадок 1,5)*

In [6]:
def get_embedding(text, emb_dict):
    
    embs = [emb_dict[w] for w in text.split() if w in emb_dict]
    
    if len(embs) == 0: 
        first_key = next(iter(emb_dict.keys()))
        return np.zeros(emb_dict[first_key].shape)
    
    return np.vstack(embs).mean(axis=0)

* *функция получения эмбеддинга документа (более подробное описание в 4ой тетрадке)*

In [7]:
def embs_predict_proba(title, text, model, emb_dict, stem, stopwords):
    
    text = preprocess(text, stem, stopwords)
    title = preprocess(title, stem, stopwords)
    
    df = pd.DataFrame({
        'title': [title], 'text': [text], 
        'title_len': [len(title)], 'text_len': [len(text)] 
    })
    
    ge = lambda s: get_embedding(s, emb_dict)
    text_series = df['title'] + ' ' + df['text']
    
    text_embs = text_series.apply(ge)
    text_embs = np.vstack(text_embs.to_numpy())

    text_len = np.expand_dims(df['text_len'].to_numpy(), axis=1)
    title_len = np.expand_dims(df['title_len'].to_numpy(), axis=1)

    X = np.hstack((text_embs, title_len, text_len))
    
    return np.round(model.predict_proba(X), 3)

* *функция получения распределения вероятностей для одного примера (модель из тетрадки 4)*

In [8]:
title = 'В декабре SpaceX, вероятно, наконец-то отправит Starship в первый орбитальный полет'
text = '''Сегодня SpaceX впервые за три года запускает ракету-носитель сверхтяжелого класса 
Falcon Heavy с миссией USSF-44, которая доставит на геостационарную орбиту 6 полезных нагрузок 
для Космических сил США (расчетное время старта — 15:41 по киевскому времени). Но есть запуск, 
которого мы ждем еще больше и уже достаточно давно. Речь идет о первом орбитальном полете 
космического корабля нового поколения Starship — в NASA предполагают, что с большой вероятностью 
запуск состоится уже в декабре этого года. По крайней мере, такова новая актуальная оценка, 
которой поделился Марк Кирасич, глава консультативного комитета пилотируемых космических полетов 
NASA и заместитель администратора по развитию амбициозной американской лунной программы 
«Артемида» / Artemis, во время недавнего заседания.'''

In [9]:
stopwords = set(
    load_lines('data/iso_stopwords_ru.txt') +
    load_lines('data/nltk_stopwords_en.txt'))

stem = pymystem3.Mystem()

* *загружаем стоп-слова и лемматайзер*

In [10]:
logreg = load_pickle('data/logreg.bin')

In [11]:
catboost = CatBoostClassifier().load_model('data/catboost.bin')

In [12]:
w2v_embs_dict = load_pickle('data/w2v_embs.bin')
w2v_embs_logreg = load_pickle('data/w2v_embs_logreg.bin')

In [13]:
tfidf_embs_dict = load_pickle('data/tfidf_embs.bin')
tfidf_embs_logreg = load_pickle('data/tfidf_embs_logreg.bin')

* *загружаем модели и эмбеддинги*

In [14]:
%time predict_proba(title, text, logreg, stem, stopwords)

CPU times: user 22.6 ms, sys: 420 µs, total: 23 ms
Wall time: 775 ms


array([[0.023, 0.002, 0.034, 0.927, 0.007, 0.007]])

* *LogReg с внутренним TFIDF векторайзером и SVD разложением*
* *самая медленная и тяжелая модель (и при этом наименее точная)*

In [15]:
%time predict_proba(title, text, catboost, stem, stopwords)

CPU times: user 14.7 ms, sys: 4.44 ms, total: 19.2 ms
Wall time: 44.1 ms


array([[0.006, 0.001, 0.079, 0.911, 0.   , 0.003]])

* *модель catboost: одна из лучших (и по скорости, и по качеству)*

In [16]:
%time embs_predict_proba(title, text, w2v_embs_logreg, w2v_embs_dict, stem, stopwords)

CPU times: user 7.89 ms, sys: 5.27 ms, total: 13.2 ms
Wall time: 25 ms


array([[0.005, 0.   , 0.011, 0.983, 0.   , 0.   ]])

* *LogReg на эмбеддингах Word2Vec: одна из самых легких и быстрых моделей*

In [17]:
%time embs_predict_proba(title, text, tfidf_embs_logreg, tfidf_embs_dict, stem, stopwords)

CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 21.6 ms


array([[0.001, 0.001, 0.041, 0.957, 0.   , 0.   ]])

* *LogReg на эмбеддингах из TFIDF: одна из самых легких и быстрых моделей*

In [18]:
list(load_json('data/labels.json').keys())

['Интернет и СМИ', 'Культура', 'Мир', 'Наука и техника', 'Спорт', 'Экономика']